In [1]:
import os
INPUT = '../input/spws-phase-1-to-2'
files = os.listdir(INPUT)

In [2]:
import pandas as pd 

tr_fs = pd.DataFrame()
te_fs = pd.DataFrame()

for _f in files:
    
    if 'train' in _f and '.csv' in _f:
        tr_fs = pd.concat([tr_fs, pd.read_csv(f'{INPUT}/{_f}')], axis=1)
    elif 'test' in _f:
        te_fs = pd.concat([te_fs, pd.read_csv(f'{INPUT}/{_f}')], axis=1)
    else:
        print(_f)

SPWS_Phase1_train_inference_train.log


In [3]:
origin = pd.read_csv('../input/signate-spws-2/train.csv')

In [4]:
origin

,id,title,abstract,judgement
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0
...,...,...,...,...
27140,27140,The amyloidogenic pathway of amyloid precursor...,Amyloid beta-protein (A beta) is the main cons...,0
27141,27141,Technologic developments in radiotherapy and s...,We present a review of current technological p...,0
27142,27142,Novel screening cascade identifies MKK4 as key...,Phosphorylation of Tau at serine 422 promotes ...,0
27143,27143,Visualization of the gall bladder on F-18 FDOP...,The ability to label dihydroxyphenylalanine (D...,0


In [5]:
border = origin['judgement'].sum(axis=0) / len(origin)
border

0.023282372444280715

In [6]:
tr_fs

,gbdt,rf,dart
0,0.000637,0.000346,0.000318
1,0.000846,0.000346,0.000521
2,0.007181,0.047323,0.012575
3,0.000733,0.000345,0.000503
4,0.000846,0.000346,0.000521
...,...,...,...
27140,0.806413,0.570002,0.848358
27141,0.001169,0.000994,0.000929
27142,0.000683,0.000518,0.000432
27143,0.001154,0.000511,0.000788


In [7]:
te_fs[te_fs.columns[0]] = te_fs[te_fs.columns[0]].apply(lambda x: 1 if x>=border else 0)
te_fs[te_fs.columns[1]] = te_fs[te_fs.columns[1]].apply(lambda x: 1 if x>=border else 0)
te_fs[te_fs.columns[2]] = te_fs[te_fs.columns[2]].apply(lambda x: 1 if x>=border else 0)


In [8]:
te_fs.mean(axis=1).round()

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
40829    0.0
40830    0.0
40831    0.0
40832    1.0
40833    0.0
Length: 40834, dtype: float64

In [9]:
sub = pd.read_csv('../input/signate-spws-2/sample_submit.csv', header=None)
sub

,0,1
0,27145,0
1,27146,1
2,27147,1
3,27148,0
4,27149,1
...,...,...
40829,67974,0
40830,67975,0
40831,67976,0
40832,67977,0


In [10]:
sub[1] = te_fs.mean(axis=1).round().astype(int)
sub.to_csv('Phase_1.csv', index=False, header=None)